In [2]:
import zipfile, os

ZIP_NAME = "drive/MyDrive/Dataset_V2.zip"  # must match the uploaded filename

with zipfile.ZipFile(ZIP_NAME, 'r') as z:
    z.extractall("/content")

print("Extracted folders in /content:")
print([p for p in os.listdir("/content") if "Dataset" in p or "dataset" in p])


Extracted folders in /content:
['Dataset_V2']


In [3]:
import os, glob

DATASET_ROOT = "/content/Dataset_V2"  # adjust if unzip created a different name

# Show top-level gesture folders
print("Top-level folders (words):")
print([d for d in os.listdir(DATASET_ROOT) if os.path.isdir(os.path.join(DATASET_ROOT, d))][:20])

# Show a few txt files
some_txt = glob.glob(os.path.join(DATASET_ROOT, "**", "*.txt"), recursive=True)
print("Found txt files:", len(some_txt))
print("Example files:", some_txt[:5])


Top-level folders (words):
['saduda', 'narakai', 'pata', 'irida', 'awidinawa', 'hawasa', 'ada', 'udasana', 'hodai', 'boru']
Found txt files: 1000
Example files: ['/content/Dataset_V2/saduda/Day_4/saduda_4.txt', '/content/Dataset_V2/saduda/Day_4/saduda_7.txt', '/content/Dataset_V2/saduda/Day_4/saduda_2.txt', '/content/Dataset_V2/saduda/Day_4/saduda_8.txt', '/content/Dataset_V2/saduda/Day_4/saduda_1.txt']


In [4]:
import numpy as np

def load_csv_robust(fp, expected_cols=10):
    """
    Reads your file safely even if it contains:
    - NULL bytes (\x00)
    - empty fields
    - trailing commas
    - corrupted rows
    Returns: float32 array (N, expected_cols)
    """
    # Read as bytes, kill nulls, decode safely
    with open(fp, "rb") as f:
        raw = f.read().replace(b"\x00", b"")  # remove NULL bytes

    text = raw.decode("utf-8", errors="ignore")

    good_rows = []
    bad = 0

    for line in text.splitlines():
        line = line.strip()
        if not line:
            continue

        # Remove trailing commas: "1,2,3," -> "1,2,3"
        while line.endswith(","):
            line = line[:-1].strip()

        parts = [p.strip() for p in line.split(",")]

        # Must have expected columns
        if len(parts) != expected_cols:
            bad += 1
            continue

        # Must not contain empty fields
        if any(p == "" for p in parts):
            bad += 1
            continue

        try:
            row = [float(p) for p in parts]
            good_rows.append(row)
        except:
            bad += 1

    if not good_rows:
        raise ValueError(f"No valid numeric rows found in {fp}. Bad rows={bad}")

    arr = np.array(good_rows, dtype=np.float32)
    return arr, bad


In [7]:
import numpy as np

fp = some_txt[0]
x = np.loadtxt(fp, delimiter=",", dtype=np.float32)
print("Example file:", fp)
print("Shape:", x.shape)   # expect (~800, 10)
print("First row:", x[0])


ValueError: could not convert string '\x006253857' to float32 at row 354, column 1.

In [5]:
import os, glob

DATASET_ROOT = "/content/Dataset_V2"

days = set()
for fp in glob.glob(os.path.join(DATASET_ROOT, "**", "*.txt"), recursive=True):
    day_name = os.path.basename(os.path.dirname(fp))   # parent folder name
    days.add(day_name)

print("Days found:", sorted(days))


Days found: ['Day_1', 'Day_10', 'Day_2', 'Day_3', 'Day_4', 'Day_5', 'Day_6', 'Day_7', 'Day_8', 'Day_9']


In [6]:
import numpy as np

def moving_average(x, w=25):
    # w=25 at 100Hz ~ 0.25s smoothing
    w = max(1, int(w))
    kernel = np.ones(w, dtype=np.float32) / w
    return np.convolve(x, kernel, mode="same")

def crop_active_region_emg(X, target_len=512, smooth_w=25, thresh_ratio=0.25):
    """
    X: (Traw, 9), EMG channels are X[:,0:3]
    Returns: (target_len, 9) by cropping around the active EMG region.

    thresh_ratio: relative threshold vs max energy (0.2~0.35 works well)
    """
    Traw = X.shape[0]
    if Traw == 0:
        return np.zeros((target_len, X.shape[1]), dtype=np.float32)

    # EMG energy signal (rectified + sum across 3 channels)
    emg = X[:, :3]
    energy = np.sum(np.abs(emg), axis=1)

    # Smooth energy so it’s stable
    energy_s = moving_average(energy, w=smooth_w)

    # Find active frames using relative threshold
    mx = float(np.max(energy_s))
    if mx <= 1e-6:
        # no activity detected -> fallback to center crop/pad
        return fix_length_center(X, target_len)

    thresh = thresh_ratio * mx
    active = np.where(energy_s >= thresh)[0]

    if len(active) < 5:
        # not enough detected activity -> fallback
        return fix_length_center(X, target_len)

    start = int(active[0])
    end   = int(active[-1])

    # Center crop window around the active segment
    center = (start + end) // 2
    half = target_len // 2
    win_start = max(0, center - half)
    win_end = win_start + target_len

    # If window goes beyond the end, shift it back
    if win_end > Traw:
        win_end = Traw
        win_start = max(0, win_end - target_len)

    cropped = X[win_start:win_end]

    # Pad if still shorter
    if cropped.shape[0] < target_len:
        pad = np.zeros((target_len - cropped.shape[0], X.shape[1]), dtype=cropped.dtype)
        cropped = np.vstack([cropped, pad])

    return cropped


In [7]:
import os, glob, json
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, classification_report

# =======================
# CONFIG (Option 1)
# =======================
DATASET_ROOT = "/content/Dataset_V2"  # <-- set correctly
T = 512   # ~8 sec @100Hz (800). 768 is convenient for pooling.
SEED = 42
EPOCHS = 60
BATCH = 32
LR = 1e-3

tf.random.set_seed(SEED)
np.random.seed(SEED)

# =======================
# DATA LOADING
# =======================
def build_label_map(root):
    labels = sorted([d for d in os.listdir(root) if os.path.isdir(os.path.join(root, d))])
    if not labels:
        raise RuntimeError(f"No class folders found in: {root}")
    return {lbl: i for i, lbl in enumerate(labels)}

def load_one_sample_txt(path):
    # CSV: t_ms, emg1, emg2, emg3, ax, ay, az, gx, gy, gz
    data = np.loadtxt(path, delimiter=",", dtype=np.float32)
    if data.ndim == 1:
        data = data[None, :]
    X = data[:, 1:]  # drop timestamp -> (Traw, 9)
    if X.shape[1] != 9:
        raise ValueError(f"{path}: expected 9 features after timestamp, got {X.shape[1]}")
    return X

def fix_length_center(X, T):
    if len(X) >= T:
        start = (len(X) - T) // 2
        return X[start:start + T]
    pad = np.zeros((T - len(X), X.shape[1]), dtype=X.dtype)
    return np.vstack([X, pad])

def emg_dc_remove(X):
    X = X.copy()
    X[:, :3] -= X[:, :3].mean(axis=0, keepdims=True)
    return X

def load_dataset(root, T):
    label2id = build_label_map(root)
    X_list, y_list = [], []
    skipped = 0

    for label, lab_id in label2id.items():
        class_dir = os.path.join(root, label)
        files = sorted(glob.glob(os.path.join(class_dir, "**", "*.txt"), recursive=True))
        for fp in files:
            try:
                X = load_one_sample_txt(fp)
                X = emg_dc_remove(X)
                X = crop_active_region_emg(X, target_len=T, smooth_w=25, thresh_ratio=0.25)
                X_list.append(X)
                y_list.append(lab_id)
            except Exception as e:
                skipped += 1
                # print("[SKIP]", fp, "->", e)

    if not X_list:
        raise RuntimeError("No samples loaded. Check DATASET_ROOT and file format.")

    X_all = np.stack(X_list, axis=0)           # (N, T, 9)
    y_all = np.array(y_list, dtype=np.int64)

    print("Loaded:", X_all.shape, "classes:", len(label2id), "skipped:", skipped)
    counts = {k: int((y_all == v).sum()) for k, v in label2id.items()}
    print("Per-class counts:", counts)

    return X_all, y_all, label2id

def normalize_train_only(X_train, X_val, X_test):
    N, T, F = X_train.shape
    scaler = StandardScaler()
    scaler.fit(X_train.reshape(-1, F))
    X_train = scaler.transform(X_train.reshape(-1, F)).reshape(N, T, F)
    X_val   = scaler.transform(X_val.reshape(-1, F)).reshape(X_val.shape[0], T, F)
    X_test  = scaler.transform(X_test.reshape(-1, F)).reshape(X_test.shape[0], T, F)
    return X_train, X_val, X_test, scaler

# =======================
# MODEL: CNN + LSTM
# =======================
def build_cnn_lstm(T, F, num_classes):
    inp = layers.Input(shape=(T, F))

    x = layers.Conv1D(64, 5, padding="same")(inp)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)
    x = layers.MaxPool1D(2)(x)

    x = layers.Conv1D(128, 3, padding="same")(x)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)
    x = layers.MaxPool1D(2)(x)

    x = layers.Dropout(0.3)(x)
    x = layers.LSTM(128)(x)
    x = layers.Dropout(0.4)(x)

    x = layers.Dense(128, activation="relu")(x)
    x = layers.Dropout(0.3)(x)

    out = layers.Dense(num_classes, activation="softmax")(x)

    model = models.Model(inp, out)
    model.compile(
        optimizer=tf.keras.optimizers.Adam(LR),
        loss="sparse_categorical_crossentropy",
        metrics=["accuracy"]
    )
    return model

def evaluate(model, X_test, y_test, id2label):
    pred = model.predict(X_test, verbose=0).argmax(axis=1)
    print("\nClassification report:")
    print(classification_report(
        y_test, pred,
        target_names=[id2label[i] for i in range(len(id2label))],
        digits=4
    ))
    cm = confusion_matrix(y_test, pred)
    print("\nConfusion matrix (rows=true, cols=pred):")
    print(cm)

# =======================
# TRAIN
# =======================
X, y, label2id = load_dataset(DATASET_ROOT, T)
id2label = {v: k for k, v in label2id.items()}

X_train, X_tmp, y_train, y_tmp = train_test_split(
    X, y, test_size=0.30, random_state=SEED, stratify=y
)
X_val, X_test, y_val, y_test = train_test_split(
    X_tmp, y_tmp, test_size=0.50, random_state=SEED, stratify=y_tmp
)

X_train, X_val, X_test, scaler = normalize_train_only(X_train, X_val, X_test)

model = build_cnn_lstm(T, X_train.shape[-1], len(label2id))
model.summary()

callbacks = [
    tf.keras.callbacks.ModelCheckpoint("/content/cnn_lstm_best.keras", monitor="val_accuracy", save_best_only=True),
    tf.keras.callbacks.EarlyStopping(monitor="val_accuracy", patience=10, restore_best_weights=True),
    tf.keras.callbacks.ReduceLROnPlateau(monitor="val_loss", factor=0.5, patience=5, min_lr=1e-6),
]

model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=EPOCHS,
    batch_size=BATCH,
    shuffle=True,
    callbacks=callbacks,
    verbose=1
)

loss, acc = model.evaluate(X_test, y_test, verbose=0)
print(f"\n✅ Test accuracy (random split): {acc:.4f}")

evaluate(model, X_test, y_test, id2label)

with open("/content/label_map.json", "w") as f:
    json.dump(label2id, f, indent=2)

print("\nSaved to /content: cnn_lstm_best.keras, label_map.json")


Loaded: (632, 512, 9) classes: 10 skipped: 368
Per-class counts: {'ada': 53, 'awidinawa': 77, 'boru': 61, 'hawasa': 58, 'hodai': 68, 'irida': 52, 'narakai': 53, 'pata': 64, 'saduda': 75, 'udasana': 71}


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 512, 9)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d (Conv1D)                 │ (None, 512, 64)        │         2,944 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 512, 64)        │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ re_lu (ReLU)                    │ (None, 512, 64)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d (MaxPooling1D)    │ (None, 256, 64)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_1 (Conv1D)               │ (None, 256, 128)       │        24,704 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 256, 128)       │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ re_lu_1 (ReLU)                  │ (None, 256, 128)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_1 (MaxPooling1D)  │ (None, 128, 128)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128, 128)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ (None, 128)            │       131,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │        16,512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 10)             │         1,290 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 177,802 (694.54 KB)

 Trainable params: 177,418 (693.04 KB)

 Non-trainable params: 384 (1.50 KB)

Epoch 1/60
14/14 ━━━━━━━━━━━━━━━━━━━━ 8s 310ms/step - accuracy: 0.1270 - loss: 2.3334 - val_accuracy: 0.2105 - val_loss: 2.2246 - learning_rate: 0.0010
Epoch 2/60
14/14 ━━━━━━━━━━━━━━━━━━━━ 5s 346ms/step - accuracy: 0.2801 - loss: 2.0808 - val_accuracy: 0.2105 - val_loss: 2.0850 - learning_rate: 0.0010
Epoch 3/60
14/14 ━━━━━━━━━━━━━━━━━━━━ 5s 392ms/step - accuracy: 0.3647 - loss: 1.8786 - val_accuracy: 0.3158 - val_loss: 1.9605 - learning_rate: 0.0010
Epoch 4/60
14/14 ━━━━━━━━━━━━━━━━━━━━ 10s 697ms/step - accuracy: 0.3772 - loss: 1.7390 - val_accuracy: 0.4316 - val_loss: 1.7744 - learning_rate: 0.0010
Epoch 5/60
14/14 ━━━━━━━━━━━━━━━━━━━━ 4s 259ms/step - accuracy: 0.3619 - loss: 1.6549 - val_accuracy: 0.4737 - val_loss: 1.6127 - learning_rate: 0.0010
Epoch 6/60
14/14 ━━━━━━━━━━━━━━━━━━━━ 4s 265ms/step - accuracy: 0.4270 - loss: 1.4402 - val_accuracy: 0.5474 - val_loss: 1.4401 - learning_rate: 0.0010
Epoch 7/60
14/14 ━━━━━━━━━━━━━━━━━━━━ 6s 311ms/step - accuracy: 0.5455 - loss: 1.2512 -

In [13]:
import os, glob, json
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, classification_report

# =======================
# CONFIG
# =======================
DATASET_ROOT = "/content/Dataset_V2"
TRAIN_DAY = "Day_1"
TEST_DAY  = "Day_2"

T = 512           # Option 2 window
SEED = 42
EPOCHS = 60
BATCH = 32
LR = 1e-3

tf.random.set_seed(SEED)
np.random.seed(SEED)

# =======================
# Robust reader (if you used it earlier)
# If your files are now clean, you can skip this and use np.loadtxt
# =======================
def load_csv_robust(fp, expected_cols=10):
    with open(fp, "rb") as f:
        raw = f.read().replace(b"\x00", b"")
    text = raw.decode("utf-8", errors="ignore")

    good_rows = []
    for line in text.splitlines():
        line = line.strip()
        if not line:
            continue
        while line.endswith(","):
            line = line[:-1].strip()
        parts = [p.strip() for p in line.split(",")]
        if len(parts) != expected_cols:
            continue
        if any(p == "" for p in parts):
            continue
        try:
            good_rows.append([float(p) for p in parts])
        except:
            continue

    if not good_rows:
        raise ValueError(f"No valid numeric rows in {fp}")
    return np.array(good_rows, dtype=np.float32)

# =======================
# Cropping (Option 2)
# =======================
def moving_average(x, w=25):
    w = max(1, int(w))
    kernel = np.ones(w, dtype=np.float32) / w
    return np.convolve(x, kernel, mode="same")

def fix_length_center(X, target_len):
    if len(X) >= target_len:
        start = (len(X) - target_len) // 2
        return X[start:start + target_len]
    pad = np.zeros((target_len - len(X), X.shape[1]), dtype=X.dtype)
    return np.vstack([X, pad])

def emg_dc_remove(X):
    X = X.copy()
    X[:, :3] -= X[:, :3].mean(axis=0, keepdims=True)
    return X

def crop_active_region_emg(X, target_len=512, smooth_w=25, thresh_ratio=0.25):
    Traw = X.shape[0]
    if Traw == 0:
        return np.zeros((target_len, X.shape[1]), dtype=np.float32)

    energy = np.sum(np.abs(X[:, :3]), axis=1)
    energy_s = moving_average(energy, w=smooth_w)

    mx = float(np.max(energy_s))
    if mx <= 1e-6:
        return fix_length_center(X, target_len)

    thresh = thresh_ratio * mx
    active = np.where(energy_s >= thresh)[0]
    if len(active) < 5:
        return fix_length_center(X, target_len)

    start = int(active[0])
    end   = int(active[-1])
    center = (start + end) // 2

    half = target_len // 2
    win_start = max(0, center - half)
    win_end = win_start + target_len
    if win_end > Traw:
        win_end = Traw
        win_start = max(0, win_end - target_len)

    cropped = X[win_start:win_end]
    if cropped.shape[0] < target_len:
        pad = np.zeros((target_len - cropped.shape[0], X.shape[1]), dtype=cropped.dtype)
        cropped = np.vstack([cropped, pad])
    return cropped

# =======================
# Dataset loading with DAY metadata
# =======================
def build_label_map(root):
    labels = sorted([d for d in os.listdir(root) if os.path.isdir(os.path.join(root, d))])
    return {lbl: i for i, lbl in enumerate(labels)}

def load_one_sample(path):
    arr = load_csv_robust(path, expected_cols=10)    # (Traw, 10)
    X = arr[:, 1:]                                   # drop timestamp -> (Traw, 9)
    X = emg_dc_remove(X)
    X = crop_active_region_emg(X, target_len=T, smooth_w=25, thresh_ratio=0.25)
    return X

def load_dataset_with_days(root):
    label2id = build_label_map(root)
    X_list, y_list, day_list = [], [], []

    for label, lab_id in label2id.items():
        class_dir = os.path.join(root, label)
        files = sorted(glob.glob(os.path.join(class_dir, "**", "*.txt"), recursive=True))

        for fp in files:
            day_name = os.path.basename(os.path.dirname(fp))  # parent folder: day_1
            try:
                X = load_one_sample(fp)
                X_list.append(X)
                y_list.append(lab_id)
                day_list.append(day_name)
            except Exception as e:
                # print("[SKIP]", fp, "->", e)
                pass

    X_all = np.stack(X_list, axis=0)  # (N, T, 9)
    y_all = np.array(y_list, dtype=np.int64)
    days  = np.array(day_list)

    print("Loaded:", X_all.shape, "classes:", len(label2id))
    print("Days present:", sorted(set(days.tolist())))
    return X_all, y_all, days, label2id

def normalize_train_only(X_train, X_val, X_test):
    N, TT, F = X_train.shape
    scaler = StandardScaler()
    scaler.fit(X_train.reshape(-1, F))

    X_train = scaler.transform(X_train.reshape(-1, F)).reshape(N, TT, F)
    X_val   = scaler.transform(X_val.reshape(-1, F)).reshape(X_val.shape[0], TT, F)
    X_test  = scaler.transform(X_test.reshape(-1, F)).reshape(X_test.shape[0], TT, F)

    return X_train, X_val, X_test, scaler


# =======================
# Model
# =======================
def build_cnn_lstm(T, F, num_classes):
    inp = layers.Input(shape=(T, F))

    x = layers.Conv1D(64, 5, padding="same")(inp)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)
    x = layers.MaxPool1D(2)(x)

    x = layers.Conv1D(128, 3, padding="same")(x)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)
    x = layers.MaxPool1D(2)(x)

    x = layers.Dropout(0.3)(x)
    x = layers.LSTM(128)(x)
    x = layers.Dropout(0.4)(x)

    x = layers.Dense(128, activation="relu")(x)
    x = layers.Dropout(0.3)(x)

    out = layers.Dense(num_classes, activation="softmax")(x)

    model = models.Model(inp, out)
    model.compile(
        optimizer=tf.keras.optimizers.Adam(LR),
        loss="sparse_categorical_crossentropy",
        metrics=["accuracy"]
    )
    return model

def evaluate(model, X_test, y_test, id2label):
    pred = model.predict(X_test, verbose=0).argmax(axis=1)
    print("\nClassification report:")
    print(classification_report(
        y_test, pred,
        target_names=[id2label[i] for i in range(len(id2label))],
        digits=4
    ))
    cm = confusion_matrix(y_test, pred)
    print("\nConfusion matrix (rows=true, cols=pred):")
    print(cm)

# =======================
# RUN: Day-wise split
# =======================
X, y, days, label2id = load_dataset_with_days(DATASET_ROOT)
id2label = {v:k for k,v in label2id.items()}

TRAIN_DAYS = {f"Day_{i}" for i in range(1, 9)}   # Day_1..Day_8
VAL_DAYS   = {"Day_9"}
TEST_DAYS  = {"Day_10"}

train_mask = np.isin(days, list(TRAIN_DAYS))
val_mask   = np.isin(days, list(VAL_DAYS))
test_mask  = np.isin(days, list(TEST_DAYS))

print("Train samples:", train_mask.sum())
print("Val samples:", val_mask.sum())
print("Test samples:", test_mask.sum())

X_train, y_train = X[train_mask], y[train_mask]
X_val,   y_val   = X[val_mask],   y[val_mask]
X_test,  y_test  = X[test_mask],  y[test_mask]

# Normalize
X_train, X_val, X_test, scaler = normalize_train_only(X_train, X_val, X_test)


model = build_cnn_lstm(T, X_train.shape[-1], len(label2id))
model.summary()

callbacks = [
    tf.keras.callbacks.EarlyStopping(monitor="val_accuracy", patience=10, restore_best_weights=True),
    tf.keras.callbacks.ReduceLROnPlateau(monitor="val_loss", factor=0.5, patience=5, min_lr=1e-6),
]

model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=EPOCHS,
    batch_size=BATCH,
    shuffle=True,
    callbacks=callbacks,
    verbose=1
)

loss, acc = model.evaluate(X_test, y_test, verbose=0)
print("✅ Test accuracy (Day_1–8 → Day_10):", acc)

evaluate(model, X_test, y_test, id2label)

# MODEL_PATH = "/content/cnn_lstm_baseline.keras"
# model.save(MODEL_PATH)
# print("Saved model to:", MODEL_PATH)

# LABEL_MAP_PATH = "/content/label_map.json"
# with open(LABEL_MAP_PATH, "w") as f:
#     json.dump(label2id, f, indent=2)

# print("Saved label map to:", LABEL_MAP_PATH)

# SCALER_PATH = "/content/scaler_params.json"
# scaler_params = {
#     "mean": scaler.mean_.tolist(),
#     "scale": scaler.scale_.tolist()
# }

# with open(SCALER_PATH, "w") as f:
#     json.dump(scaler_params, f, indent=2)

# print("Saved scaler params to:", SCALER_PATH)



# Force-save model in TF-compatible HDF5 format
model.save("cnn_lstm_baseline.h5", save_format="h5")
print("Saved cnn_lstm_baseline.h5")

# Also resave label map + scaler (just to be safe)
import json

with open("label_map.json", "w") as f:
    json.dump(label2id, f, indent=2)

scaler_params = {
    "mean": scaler.mean_.tolist(),
    "scale": scaler.scale_.tolist()
}
with open("scaler_params.json", "w") as f:
    json.dump(scaler_params, f, indent=2)

# Save as TensorFlow SavedModel (folder)
model.export("/content/cnn_lstm_savedmodel")
print("Saved SavedModel to /content/cnn_lstm_savedmodel")

!zip -r cnn_lstm_savedmodel.zip /content/cnn_lstm_savedmodel




Loaded: (1000, 512, 9) classes: 10
Days present: ['Day_1', 'Day_10', 'Day_2', 'Day_3', 'Day_4', 'Day_5', 'Day_6', 'Day_7', 'Day_8', 'Day_9']
Train samples: 801
Val samples: 100
Test samples: 99


Model: "functional_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_3 (InputLayer)      │ (None, 512, 9)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_6 (Conv1D)               │ (None, 512, 64)        │         2,944 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_6           │ (None, 512, 64)        │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ re_lu_6 (ReLU)                  │ (None, 512, 64)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_6 (MaxPooling1D)  │ (None, 256, 64)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_7 (Conv1D)               │ (None, 256, 128)       │        24,704 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_7           │ (None, 256, 128)       │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ re_lu_7 (ReLU)                  │ (None, 256, 128)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_7 (MaxPooling1D)  │ (None, 128, 128)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_9 (Dropout)             │ (None, 128, 128)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_3 (LSTM)                   │ (None, 128)            │       131,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_10 (Dropout)            │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 128)            │        16,512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_11 (Dropout)            │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 10)             │         1,290 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 177,802 (694.54 KB)

 Trainable params: 177,418 (693.04 KB)

 Non-trainable params: 384 (1.50 KB)

Epoch 1/60
26/26 ━━━━━━━━━━━━━━━━━━━━ 17s 276ms/step - accuracy: 0.1441 - loss: 2.2968 - val_accuracy: 0.1800 - val_loss: 2.1508 - learning_rate: 0.0010
Epoch 2/60
26/26 ━━━━━━━━━━━━━━━━━━━━ 9s 236ms/step - accuracy: 0.2720 - loss: 1.9828 - val_accuracy: 0.1800 - val_loss: 2.0470 - learning_rate: 0.0010
Epoch 3/60
26/26 ━━━━━━━━━━━━━━━━━━━━ 10s 238ms/step - accuracy: 0.3682 - loss: 1.8090 - val_accuracy: 0.3300 - val_loss: 1.6851 - learning_rate: 0.0010
Epoch 4/60
26/26 ━━━━━━━━━━━━━━━━━━━━ 12s 305ms/step - accuracy: 0.3925 - loss: 1.6041 - val_accuracy: 0.4900 - val_loss: 1.4124 - learning_rate: 0.0010
Epoch 5/60
26/26 ━━━━━━━━━━━━━━━━━━━━ 6s 236ms/step - accuracy: 0.4570 - loss: 1.3971 - val_accuracy: 0.4800 - val_loss: 1.2977 - learning_rate: 0.0010
Epoch 6/60
26/26 ━━━━━━━━━━━━━━━━━━━━ 8s 300ms/step - accuracy: 0.5894 - loss: 1.1464 - val_accuracy: 0.3900 - val_loss: 1.6628 - learning_rate: 0.0010
Epoch 7/60
26/26 ━━━━━━━━━━━━━━━━━━━━ 6s 233ms/step - accuracy: 0.6226 - loss: 1.0394


Classification report:
              precision    recall  f1-score   support

         ada     1.0000    0.5000    0.6667        10
   awidinawa     0.9000    0.9000    0.9000        10
        boru     1.0000    0.9000    0.9474        10
      hawasa     1.0000    1.0000    1.0000        10
       hodai     0.7143    1.0000    0.8333        10
       irida     1.0000    1.0000    1.0000         9
     narakai     0.9091    1.0000    0.9524        10
        pata     0.7692    1.0000    0.8696        10
      saduda     0.9091    1.0000    0.9524        10
     udasana     1.0000    0.7000    0.8235        10

    accuracy                         0.8990        99
   macro avg     0.9202    0.9000    0.8945        99
weighted avg     0.9194    0.8990    0.8935        99


Confusion matrix (rows=true, cols=pred):
[[ 5  1  0  0  4  0  0  0  0  0]
 [ 0  9  0  0  0  0  1  0  0  0]
 [ 0  0  9  0  0  0  0  0  1  0]
 [ 0  0  0 10  0  0  0  0  0  0]
 [ 0  0  0  0 10  0  0  0  0  0]
 [ 0  0  0

In [10]:
# Save as TensorFlow SavedModel (folder)
model.export("/content/cnn_lstm_savedmodel")
print("Saved SavedModel to /content/cnn_lstm_savedmodel")


Saved artifact at '/content/cnn_lstm_savedmodel'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 512, 9), dtype=tf.float32, name='keras_tensor_30')
Output Type:
  TensorSpec(shape=(None, 10), dtype=tf.float32, name=None)
Captures:
  139333017091344: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139333017091152: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139332880254160: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139332880254352: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139333017091728: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139333017091536: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139332880253008: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139332880253584: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139332880253968: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139332880255504: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139

In [11]:
!zip -r cnn_lstm_savedmodel.zip /content/cnn_lstm_savedmodel


  adding: content/cnn_lstm_savedmodel/ (stored 0%)
  adding: content/cnn_lstm_savedmodel/saved_model.pb (deflated 86%)
  adding: content/cnn_lstm_savedmodel/variables/ (stored 0%)
  adding: content/cnn_lstm_savedmodel/variables/variables.data-00000-of-00001 (deflated 8%)
  adding: content/cnn_lstm_savedmodel/variables/variables.index (deflated 68%)
  adding: content/cnn_lstm_savedmodel/assets/ (stored 0%)
  adding: content/cnn_lstm_savedmodel/fingerprint.pb (stored 0%)


In [12]:
m = tf.keras.models.load_model("/content/cnn_lstm_best.keras", compile=False)
m.export("/content/cnn_lstm_savedmodel")


Saved artifact at '/content/cnn_lstm_savedmodel'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): List[TensorSpec(shape=(None, 512, 9), dtype=tf.float32, name='input_layer')]
Output Type:
  TensorSpec(shape=(None, 10), dtype=tf.float32, name=None)
Captures:
  139332880260304: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139332880262416: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139332880260880: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139332880256848: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139332880254544: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139332880260496: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139332880259152: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139332876746576: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139332876737552: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139332876745424: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1